In [1]:
import speech_recognition as srec
from gtts import gTTS
import pyttsx3 as pyt
import os

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, VitsModel
import torch

import time
import psutil

import whisper
import sounddevice as sd
import numpy as np
from IPython.display import Audio

c:\Users\Alysha\Documents\kata-ondevice\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
engine = pyt.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

device = "auto"

In [ ]:
# --- AZURE SAILOR 0.5B MODEL ---
start_time = time.time()

model = AutoModelForCausalLM.from_pretrained(
    'sail/Sailor-0.5B-Chat',
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained('sail/Sailor-0.5B-Chat')
end_time = time.time()
load_time = end_time - start_time
print(load_time)


In [2]:
# --- LLAMA ---

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", torch_dtype=torch.bfloat16, 
  device_map='cpu')

c:\Users\Alysha\Documents\kata-ondevice\myenv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alysha\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
# --- SeaLLMs-v3-7B-Chat ---

model = AutoModelForCausalLM.from_pretrained(
 "SeaLLMs/SeaLLMs-v3-1.5B-Chat", # can change to "SeaLLMs/SeaLLMs-v3-1.5B-Chat" if your resource is limited
  torch_dtype=torch.bfloat16, 
  device_map='cpu'
)
tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLMs-v3-1.5B-Chat")

In [3]:
# --- STT ---

# def perintah():
#     mendengar = srec.Recognizer()
#     with srec.Microphone() as source:
#         print('Mendengarkan......')
#         suara = mendengar.listen(source, phrase_time_limit=5)
#         try:
#             print('Diterima.....')
#             dengar = mendengar.recognize_google(suara, language='id-ID')
#             print(dengar)
#         except:
#             pass
#         return dengar

whisper_model = whisper.load_model("small")

def perintah():
    duration = 5
    sample_rate = 16000  

    print("Mendengarkan......")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  
    print("Diterima.....")

    audio_data = np.squeeze(audio_data)  
    dengar = whisper_model.transcribe(audio_data, fp16=False, language="id")
    print(dengar["text"])

    return dengar["text"]

c:\Users\Alysha\Documents\kata-ondevice\myenv\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locati

In [5]:
# --- TTS ---

def ngomong(text):
    voices = engine.getProperty('voices')

    for voice in voices:
        if "MSTTS_V110_idID_Andika" in voice.id:
            engine.setProperty('voice', voice.id)
            break

    # Speak the text
    engine.say(text)
    
    # Wait until speaking is finished
    engine.runAndWait()

# mms = VitsModel.from_pretrained("facebook/mms-tts-ind")
# mms_token = AutoTokenizer.from_pretrained("facebook/mms-tts-ind")

# def ngomong(text):
#     inputs = mms_token(text, return_tensors="pt")

#     with torch.no_grad():
#         output = mms(**inputs).waveform
    
#     return Audio(output.squeeze().cpu().numpy(), rate=16000) # Can change rate to make it faster/slower

In [6]:
def run_va():
    # prompt from user
    Layanan = perintah()

    # messages = [{"role": "user", "content": Layanan}]
    # prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # print(outputs[0]["generated_text"])

    # ngomong(Layanan)

    # print(Layanan)

    system_prompt= 'Jawab dalam bahasa indonesia.'

    # messages = [
    #     {"role": "question", "content": Layanan}
    #     ]
    messages = [
    {"role": "system", "content": "You are a helpful assistant. "},
    {"role": "user", "content": Layanan}
]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt") 

    start_time = time.time()
    memory_before = psutil.virtual_memory().used

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512, 
        temperature=0.7,
        top_p=0.9,
        do_sample=True

    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    end_time = time.time()
    memory_after = psutil.virtual_memory().used

    inference_time = end_time - start_time
    memory_used = memory_after - memory_before
    cpu_usage = psutil.cpu_percent(interval=1)
    
    print(response)
    print("INFERENCE TIME: ", inference_time)
    print("MEMORY USAGE: ", memory_used)
    print("CPU USAGE: ", cpu_usage)
    ngomong(response)

In [13]:
run_va()

Mendengarkan......
Diterima.....


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 Apa ibu kota Indonesia?
Jakarta
INFERENCE TIME:  42.532082319259644
MEMORY USAGE:  718254080
CPU USAGE:  8.3


In [ ]:
ngomong("Selamat malam, Francesco. Nama saya Budi.")